In [1]:
import os
import sys
sys.path.append('../../..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import joblib
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing
from sklearn.utils import shuffle
from tensorflow.keras.models import save_model,load_model

from utils.util import *
from utils.preprocessing import *
from utils.dataiter import Dataiter
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score
from utils.target_encode import MTE_one_shot

import tensorflow.keras.backend as K
import core.config as conf

## Load Data

In [2]:
path = f'{conf.dataset_mini_path}/train'
train = read_data(path)

path = f'{conf.dataset_mini_path}/test'
test = read_data(path)

path = f'{conf.dataset_mini_path}/valid'
valid = read_data(path)

In [3]:
TARGET = 'reply'

## Preprocessing

In [4]:
def set_dataframe_types(df, train):
    df['id']   = np.arange( df.shape[0] )
    df['id']   = df['id'].astype(np.uint32)

    if train:
        df['reply_timestamp']   = df['reply_timestamp'].fillna(0)
        df['retweet_timestamp'] = df['retweet_timestamp'].fillna(0)
        df['comment_timestamp'] = df['comment_timestamp'].fillna(0)
        df['like_timestamp']    = df['like_timestamp'].fillna(0)

        df['reply_timestamp']   = df['reply_timestamp'].astype(np.uint32)
        df['retweet_timestamp'] = df['retweet_timestamp'].astype(np.uint32)
        df['comment_timestamp'] = df['comment_timestamp'].astype(np.uint32)
        df['like_timestamp']    = df['like_timestamp'].astype(np.uint32)

    df['tweet_timestamp']         = df['tweet_timestamp'].astype( np.uint32 )
    df['creator_follower_count']  = df['creator_follower_count'].astype( np.uint32 )
    df['creator_following_count'] = df['creator_following_count'].astype( np.uint32 )
    df['creator_account_creation']= df['creator_account_creation'].astype( np.uint32 )
    df['engager_follower_count']  = df['engager_follower_count'].astype( np.uint32 )
    df['engager_following_count'] = df['engager_following_count'].astype( np.uint32 )
    df['engager_account_creation']= df['engager_account_creation'].astype( np.uint32 )

    return df



def preprocess(df, target, train):
    df = set_dataframe_types(df, train)
    # df = df.set_index('id')
    # df.columns = conf.raw_features + conf.labels
    df = df.drop('text_tokens', axis=1)
    
    df = feature_extraction(df, features=conf.used_features, train=train) # extract 'used_features'
    cols = []

    return df

In [5]:
train = preprocess(train, TARGET, True)
valid = preprocess(valid, TARGET, True)
test = preprocess(test, TARGET, True)

In [6]:
train

,creator_id,engager_id,tweet_id,tweet_type,language,creator_account_creation,creator_follower_count,creator_following_count,engager_follower_count,engager_following_count,...,media,tweet_timestamp,reply,retweet,comment,like,dt_day,dt_dow,dt_hour,len_domains
0,F09233A58769507FD4E6FD618BCFA5B6,CFFD4D0D3AAF28B5397A98E1F4C92C77,6238B9E15E83B6D477394E9D80B3784E,3,7F4FAB1EB12CD95EDCD9DB2A6634EFCE,1525021638,560,645,595,1424,...,4,1612883086,0.0,0.0,0.0,0.0,9,1,15,0
1,46BEEE566BB708F18075A60FDDAA8503,E0F9A3BB331C7B613B165F32F8D273BE,731FB90C6CFEF1B71D322106573F71DB,0,E7F038DE3EAD397AEC9193686C911677,1348805977,2591,3690,514,416,...,1,1612864776,0.0,0.0,0.0,0.0,9,1,9,0
2,83F59BF2E0778AC4078FA3F7B71F5960,58ABF7FB24D65FB94C6487915A889460,89276E5272498E858EE8AF691EBF0951,3,1F73BB863A39DB62B4A55B7E558DB1E8,1583939938,174,146,376,311,...,0,1613661020,0.0,0.0,0.0,1.0,18,3,15,0
3,C96110509F8B6256CE08A8AE85ED75DB,76730357A84D1D04A662FDB1449F1AFF,502FDBC0EB4E7AB157D38262817716EB,3,310ECD7D1E42216E3C1B31EFDDFC72A7,1306675315,115852,2124,22461,16342,...,0,1612751908,0.0,0.0,0.0,0.0,8,0,2,1
4,2031F18622E7C6A30E702D33776BF2C1,9008C5AB3CB46623ADBD4C68A2A2D088,CF1F523F7D4D4139E5FD3EBD72F27D5A,3,488B32D24BD4BB44172EB981C1BCA6FA,1308377880,1208919,1101,388,647,...,0,1613060168,0.0,0.0,0.0,1.0,11,3,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4338901,968FAF2F52AACA58C2B41CE5B13A811D,1BC672DA2F063F2DE03DA89BA6B621C5,EE63068E74D7E293457FDC2EA3527D65,3,E7F038DE3EAD397AEC9193686C911677,1359315180,248,315,1785,1908,...,1,1612670742,0.0,0.0,0.0,1.0,7,6,4,0
4338902,54652625625729AE0E38196970594938,39D3029E8FDC3A18472D28436EC9E776,F15B4615DA12447331AFEC90FA814B4D,0,488B32D24BD4BB44172EB981C1BCA6FA,1293912925,114,937,183,713,...,0,1612744473,0.0,0.0,0.0,0.0,8,0,0,0
4338903,614E269F8E5D731946BDBC86E9DC4F4D,4A4E526A7390A5CC9D02ECB5CB62F377,DA947A6368CF4414BEA9172003A639FD,3,B8B04128918BBF54E2E178BFF1ABA833,1512095330,56033,121,58,48,...,4,1612397867,0.0,0.0,0.0,1.0,4,3,0,0
4338904,5A5C7CCA69CDEBBD7E81C3797DE9DD7E,D8CAFE1402CE86E007370664FC553C2B,730879D11261FA6700804AD49CCB59BB,3,488B32D24BD4BB44172EB981C1BCA6FA,1251066468,6745,368,2,135,...,4,1612956173,0.0,0.0,0.0,1.0,10,2,11,0


### pickle matching

#### language

In [7]:
pickle_path = conf.dict_path

In [8]:
user_main_language_path = pickle_path + "user_main_language.pkl"

if os.path.exists(user_main_language_path) :
    with open(user_main_language_path, 'rb') as f :
        user_main_language = pickle.load(f)

In [9]:
language_dict_path = pickle_path + "language_dict.pkl"

if os.path.exists(language_dict_path ) :
    with open(language_dict_path , 'rb') as f :
        language_dict = pickle.load(f)

In [10]:
train['language'] = train.apply(lambda x : language_dict[x['language']], axis = 1)
test['language'] = test.apply(lambda x : language_dict[x['language']], axis = 1)
valid['language'] = valid.apply(lambda x : language_dict[x['language']], axis = 1)

In [11]:
del language_dict

In [12]:
train['creator_main_language'] = train['creator_id'].map(user_main_language)
valid['creator_main_language'] = valid['creator_id'].map(user_main_language)
test['creator_main_language'] = test['creator_id'].map(user_main_language)

In [13]:
train['engager_main_language'] = train['engager_id'].map(user_main_language)
valid['engager_main_language'] = valid['engager_id'].map(user_main_language)
test['engager_main_language'] = test['engager_id'].map(user_main_language)

In [14]:
train['creator_and_engager_have_same_main_language'] = train.apply(lambda x : 1 if x['creator_main_language'] == x['engager_main_language'] else 0, axis = 1)
valid['creator_and_engager_have_same_main_language'] = valid.apply(lambda x : 1 if x['creator_main_language'] == x['engager_main_language'] else 0, axis = 1)
test['creator_and_engager_have_same_main_language'] = test.apply(lambda x : 1 if x['creator_main_language'] == x['engager_main_language'] else 0, axis = 1)

In [15]:
train['is_tweet_in_creator_main_language'] = train.apply(lambda x : 1 if x['creator_main_language'] == x['language'] else 0, axis = 1)
valid['is_tweet_in_creator_main_language'] = valid.apply(lambda x : 1 if x['creator_main_language'] == x['language'] else 0, axis = 1)
test['is_tweet_in_creator_main_language'] = test.apply(lambda x : 1 if x['creator_main_language'] == x['language'] else 0, axis = 1)    

In [16]:
train['is_tweet_in_engager_main_language'] = train.apply(lambda x : 1 if x['engager_main_language'] == x['language'] else 0, axis = 1)
valid['is_tweet_in_engager_main_language'] = valid.apply(lambda x : 1 if x['engager_main_language'] == x['language'] else 0, axis = 1)
test['is_tweet_in_engager_main_language'] = test.apply(lambda x : 1 if x['engager_main_language'] == x['language'] else 0, axis = 1)

In [17]:
del user_main_language

In [18]:
train.head()

,creator_id,engager_id,tweet_id,tweet_type,language,creator_account_creation,creator_follower_count,creator_following_count,engager_follower_count,engager_following_count,...,like,dt_day,dt_dow,dt_hour,len_domains,creator_main_language,engager_main_language,creator_and_engager_have_same_main_language,is_tweet_in_creator_main_language,is_tweet_in_engager_main_language
0,F09233A58769507FD4E6FD618BCFA5B6,CFFD4D0D3AAF28B5397A98E1F4C92C77,6238B9E15E83B6D477394E9D80B3784E,3,10,1525021638,560,645,595,1424,...,0.0,9,1,15,0,9,35,0,0,0
1,46BEEE566BB708F18075A60FDDAA8503,E0F9A3BB331C7B613B165F32F8D273BE,731FB90C6CFEF1B71D322106573F71DB,0,1,1348805977,2591,3690,514,416,...,0.0,9,1,9,0,1,1,1,1,1
2,83F59BF2E0778AC4078FA3F7B71F5960,58ABF7FB24D65FB94C6487915A889460,89276E5272498E858EE8AF691EBF0951,3,5,1583939938,174,146,376,311,...,1.0,18,3,15,0,5,4,0,1,0
3,C96110509F8B6256CE08A8AE85ED75DB,76730357A84D1D04A662FDB1449F1AFF,502FDBC0EB4E7AB157D38262817716EB,3,12,1306675315,115852,2124,22461,16342,...,0.0,8,0,2,1,12,1,0,1,0
4,2031F18622E7C6A30E702D33776BF2C1,9008C5AB3CB46623ADBD4C68A2A2D088,CF1F523F7D4D4139E5FD3EBD72F27D5A,3,0,1308377880,1208919,1101,388,647,...,1.0,11,3,16,0,0,13,0,1,0


#### engagements

In [19]:
engagement_like_path = pickle_path + "engagement-like.pkl"

if os.path.exists(engagement_like_path ) :
    with open(engagement_like_path , 'rb') as f :
        engagement_like = pickle.load(f)

In [20]:
train['engager_feature_number_of_previous_like_engagement'] = train.apply(lambda x : engagement_like[x['engager_id']], axis = 1)
valid['engager_feature_number_of_previous_like_engagement'] = valid.apply(lambda x : engagement_like[x['engager_id']], axis = 1)
test['engager_feature_number_of_previous_like_engagement'] = test.apply(lambda x : engagement_like[x['engager_id']], axis = 1)



In [21]:
engagement_like_path = pickle_path + "creator-engagement-like.pkl"

if os.path.exists(engagement_like_path ) :
    with open(engagement_like_path , 'rb') as f :
        creator_engagement_like = pickle.load(f)

train['creator_feature_number_of_previous_like_engagement'] = train.apply(lambda x : creator_engagement_like[x['creator_id']], axis = 1)
valid['creator_feature_number_of_previous_like_engagement'] = valid.apply(lambda x : creator_engagement_like[x['creator_id']], axis = 1)
test['creator_feature_number_of_previous_like_engagement'] = test.apply(lambda x : creator_engagement_like[x['creator_id']], axis = 1)

In [22]:
del engagement_like
del creator_engagement_like

In [23]:
engagement_reply_path = pickle_path + "engagement-reply.pkl"

if os.path.exists(engagement_reply_path ) :
    with open(engagement_reply_path , 'rb') as f :
        engagement_reply = pickle.load(f)

In [24]:
train['engager_feature_number_of_previous_reply_engagement'] = train.apply(lambda x : engagement_reply[x['engager_id']], axis = 1)
valid['engager_feature_number_of_previous_reply_engagement'] = valid.apply(lambda x : engagement_reply[x['engager_id']], axis = 1)
test['engager_feature_number_of_previous_reply_engagement'] = test.apply(lambda x : engagement_reply[x['engager_id']], axis = 1)


In [25]:
engagement_reply_path = pickle_path + "creator-engagement-reply.pkl"

if os.path.exists(engagement_reply_path ) :
    with open(engagement_reply_path , 'rb') as f :
        creator_engagement_reply = pickle.load(f)
        
train['creator_feature_number_of_previous_reply_engagement'] = train.apply(lambda x : creator_engagement_reply[x['creator_id']], axis = 1)
valid['creator_feature_number_of_previous_reply_engagement'] = valid.apply(lambda x : creator_engagement_reply[x['creator_id']], axis = 1)
test['creator_feature_number_of_previous_reply_engagement'] = test.apply(lambda x : creator_engagement_reply[x['creator_id']], axis = 1)

In [26]:
del engagement_reply
del creator_engagement_reply

In [27]:
engagement_retweet_path = pickle_path + "engagement-retweet.pkl"

if os.path.exists(engagement_retweet_path ) :
    with open(engagement_retweet_path , 'rb') as f :
        engagement_retweet = pickle.load(f)

In [28]:
train['engager_feature_number_of_previous_retweet_engagement'] = train.apply(lambda x : engagement_retweet[x['engager_id']], axis = 1)
valid['engager_feature_number_of_previous_retweet_engagement'] = valid.apply(lambda x : engagement_retweet[x['engager_id']], axis = 1)
test['engager_feature_number_of_previous_retweet_engagement'] = test.apply(lambda x : engagement_retweet[x['engager_id']], axis = 1)


In [29]:
engagement_retweet_path = pickle_path + "creator-engagement-retweet.pkl"

if os.path.exists(engagement_retweet_path ) :
    with open(engagement_retweet_path , 'rb') as f :
        creator_engagement_retweet = pickle.load(f)

train['creator_feature_number_of_previous_retweet_engagement'] = train.apply(lambda x : creator_engagement_retweet[x['creator_id']], axis = 1)
valid['creator_feature_number_of_previous_retweet_engagement'] = valid.apply(lambda x : creator_engagement_retweet[x['creator_id']], axis = 1)
test['creator_feature_number_of_previous_retweet_engagement'] = test.apply(lambda x : creator_engagement_retweet[x['creator_id']], axis = 1)


In [30]:
del engagement_retweet
del creator_engagement_retweet

In [31]:
engagement_comment_path = pickle_path + "engagement-comment.pkl"

if os.path.exists(engagement_comment_path ) :
    with open(engagement_comment_path , 'rb') as f :
        engagement_comment = pickle.load(f)

In [32]:
train['engager_feature_number_of_previous_comment_engagement'] = train.apply(lambda x : engagement_comment[x['engager_id']], axis = 1)
valid['engager_feature_number_of_previous_comment_engagement'] = valid.apply(lambda x : engagement_comment[x['engager_id']], axis = 1)
test['engager_feature_number_of_previous_comment_engagement'] = test.apply(lambda x : engagement_comment[x['engager_id']], axis = 1)



In [33]:
engagement_comment_path = pickle_path + "creator-engagement-comment.pkl"

if os.path.exists(engagement_comment_path ) :
    with open(engagement_comment_path , 'rb') as f :
        creator_engagement_comment = pickle.load(f)

train['creator_feature_number_of_previous_comment_engagement'] = train.apply(lambda x : creator_engagement_comment[x['creator_id']], axis = 1)
valid['creator_feature_number_of_previous_comment_engagement'] = valid.apply(lambda x : creator_engagement_comment[x['creator_id']], axis = 1)
test['creator_feature_number_of_previous_comment_engagement'] = test.apply(lambda x : creator_engagement_comment[x['creator_id']], axis = 1)


In [34]:
del engagement_comment
del creator_engagement_comment

In [35]:
train['number_of_engagements_positive'] = train.apply(lambda x : x['engager_feature_number_of_previous_like_engagement'] + x['engager_feature_number_of_previous_retweet_engagement'] + x['engager_feature_number_of_previous_reply_engagement'] + x['engager_feature_number_of_previous_comment_engagement'], axis = 1)
valid['number_of_engagements_positive'] = valid.apply(lambda x : x['engager_feature_number_of_previous_like_engagement'] + x['engager_feature_number_of_previous_retweet_engagement'] + x['engager_feature_number_of_previous_reply_engagement'] + x['engager_feature_number_of_previous_comment_engagement'], axis = 1)
test['number_of_engagements_positive'] = test.apply(lambda x : x['engager_feature_number_of_previous_like_engagement'] + x['engager_feature_number_of_previous_retweet_engagement'] + x['engager_feature_number_of_previous_reply_engagement'] + x['engager_feature_number_of_previous_comment_engagement'], axis = 1)

In [36]:
train['creator_number_of_engagements_positive'] = train.apply(lambda x : x['creator_feature_number_of_previous_like_engagement'] + x['creator_feature_number_of_previous_retweet_engagement'] + x['creator_feature_number_of_previous_reply_engagement'] + x['creator_feature_number_of_previous_comment_engagement'], axis = 1)
valid['creator_number_of_engagements_positive'] = valid.apply(lambda x : x['creator_feature_number_of_previous_like_engagement'] + x['creator_feature_number_of_previous_retweet_engagement'] + x['creator_feature_number_of_previous_reply_engagement'] + x['creator_feature_number_of_previous_comment_engagement'], axis = 1)
test['creator_number_of_engagements_positive'] = test.apply(lambda x : x['creator_feature_number_of_previous_like_engagement'] + x['creator_feature_number_of_previous_retweet_engagement'] + x['creator_feature_number_of_previous_reply_engagement'] + x['creator_feature_number_of_previous_comment_engagement'], axis = 1)

In [37]:
# train = train.drop('engager_feature_number_of_previous_reply_engagement', axis = 1)
train = train.drop('engager_feature_number_of_previous_like_engagement', axis = 1)
train = train.drop('engager_feature_number_of_previous_retweet_engagement', axis = 1)
train = train.drop('engager_feature_number_of_previous_comment_engagement', axis = 1)

In [38]:
# train = train.drop('engager_feature_number_of_previous_reply_engagement', axis = 1)
train = train.drop('creator_feature_number_of_previous_like_engagement', axis = 1)
train = train.drop('creator_feature_number_of_previous_retweet_engagement', axis = 1)
train = train.drop('creator_feature_number_of_previous_comment_engagement', axis = 1)

In [39]:
# valid = valid.drop('engager_feature_number_of_previous_reply_engagement', axis = 1)
valid = valid.drop('engager_feature_number_of_previous_like_engagement', axis = 1)
valid = valid.drop('engager_feature_number_of_previous_retweet_engagement', axis = 1)
valid = valid.drop('engager_feature_number_of_previous_comment_engagement', axis = 1)

In [40]:
# valid = valid.drop('creator_feature_number_of_previous_reply_engagement', axis = 1)
valid = valid.drop('creator_feature_number_of_previous_like_engagement', axis = 1)
valid = valid.drop('creator_feature_number_of_previous_retweet_engagement', axis = 1)
valid = valid.drop('creator_feature_number_of_previous_comment_engagement', axis = 1)

In [41]:
# test = test.drop('engager_feature_number_of_previous_reply_engagement', axis = 1)
test = test.drop('engager_feature_number_of_previous_like_engagement', axis = 1)
test = test.drop('engager_feature_number_of_previous_retweet_engagement', axis = 1)
test = test.drop('engager_feature_number_of_previous_comment_engagement', axis = 1)

In [42]:
# test = test.drop('creator_feature_number_of_previous_reply_engagement', axis = 1)
test = test.drop('creator_feature_number_of_previous_like_engagement', axis = 1)
test = test.drop('creator_feature_number_of_previous_retweet_engagement', axis = 1)
test = test.drop('creator_feature_number_of_previous_comment_engagement', axis = 1)

In [43]:
# train['number_of_engagements_ratio_like'] = train.apply(lambda x : x['engager_feature_number_of_previous_like_engagement'] / x['number_of_engagements_positive'] if x['number_of_engagements_positive'] != 0 else 0, axis = 1)
# valid['number_of_engagements_ratio_like'] = valid.apply(lambda x : x['engager_feature_number_of_previous_like_engagement'] / x['number_of_engagements_positive'] if x['number_of_engagements_positive'] != 0 else 0, axis = 1)
# test['number_of_engagements_ratio_like'] = test.apply(lambda x : x['engager_feature_number_of_previous_like_engagement'] / x['number_of_engagements_positive'] if x['number_of_engagements_positive'] != 0 else 0, axis = 1)

In [44]:
train[f'engager_number_of_engagements_ratio_{TARGET}'] = train.apply(lambda x : x[f'engager_feature_number_of_previous_{TARGET}_engagement'] / x['number_of_engagements_positive'] if x['number_of_engagements_positive'] != 0 else 0, axis = 1)
valid[f'engager_number_of_engagements_ratio_{TARGET}'] = valid.apply(lambda x : x[f'engager_feature_number_of_previous_{TARGET}_engagement'] / x['number_of_engagements_positive'] if x['number_of_engagements_positive'] != 0 else 0, axis = 1)
test[f'engager_number_of_engagements_ratio_{TARGET}'] = test.apply(lambda x : x[f'engager_feature_number_of_previous_{TARGET}_engagement'] / x['number_of_engagements_positive'] if x['number_of_engagements_positive'] != 0 else 0, axis = 1)

In [48]:
train[f'creator_number_of_engagements_ratio_{TARGET}'] = train.apply(lambda x : x[f'creator_feature_number_of_previous_{TARGET}_engagement'] / x['creator_number_of_engagements_positive'] if x['creator_number_of_engagements_positive'] != 0 else 0, axis = 1)
valid[f'creator_number_of_engagements_ratio_{TARGET}'] = valid.apply(lambda x : x[f'creator_feature_number_of_previous_{TARGET}_engagement'] / x['creator_number_of_engagements_positive'] if x['creator_number_of_engagements_positive'] != 0 else 0, axis = 1)
test[f'creator_number_of_engagements_ratio_{TARGET}'] = test.apply(lambda x : x[f'creator_feature_number_of_previous_{TARGET}_engagement'] / x['creator_number_of_engagements_positive'] if x['creator_number_of_engagements_positive'] != 0 else 0, axis = 1)

## Sampling

In [46]:
# df_positive = train[train[TARGET]==1]
# df_negative = train[train[TARGET]==0]
# print(len(df_positive))
# print(len(df_negative))

In [47]:
# df_negative = df_negative.sample(n = len(df_positive), random_state=777)

In [48]:
# train = pd.concat([df_positive, df_negative])

In [49]:
# train  = train.sample(frac = 1)

In [50]:
# del df_positive
# del df_negative

## Split

In [49]:
label_names = ['reply', 'retweet', 'comment', 'like']
DONT_USE = ['tweet_timestamp','creator_account_creation','engager_account_creation','engage_time',
            'creator_account_creation', 'engager_account_creation',
            'fold','tweet_id', 
            'tr','dt_day','','',
            'engager_id','creator_id','engager_is_verified',
            'elapsed_time',
            'links','domains','hashtags0','hashtags1',
            'hashtags','tweet_hash','dt_second','id',
            'tw_hash0',
            'tw_hash1',
            'tw_rt_uhash',
            'same_language', 'nan_language','language',
            'tw_hash', 'tw_freq_hash','tw_first_word', 'tw_second_word', 'tw_last_word', 'tw_llast_word',
            'ypred','creator_count_combined','creator_user_fer_count_delta_time','creator_user_fing_count_delta_time','creator_user_fering_count_delta_time','creator_user_fing_count_mode','creator_user_fer_count_mode','creator_user_fering_count_mode'
           ]
DONT_USE += label_names
DONT_USE += conf.labels

In [50]:
RMV = [c for c in DONT_USE if c in train.columns]

In [51]:
y_train = train[TARGET]
X_train = train.drop(RMV, axis=1)

In [52]:
del train

In [53]:
y_valid = valid[TARGET]
X_valid = valid.drop(RMV, axis=1)
del valid

In [54]:
y_test = test[TARGET]
X_test = test.drop(RMV, axis=1)
del test

## Scaling

In [55]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_val = X_valid.reset_index(drop=True)

In [62]:
scaling_columns = ['creator_following_count', 'creator_follower_count', 'engager_follower_count', 'engager_following_count', f'engager_feature_number_of_previous_{TARGET}_engagement',f'creator_feature_number_of_previous_{TARGET}_engagement', 'number_of_engagements_positive', 'creator_number_of_engagements_positive', 'dt_dow', 'dt_hour', 'len_domains']

In [63]:
standard_scaler = preprocessing.StandardScaler()
standard_scaler.fit(X_train[scaling_columns])

StandardScaler()

In [64]:
ss = standard_scaler.transform(X_train[scaling_columns])
X_train[scaling_columns] = pd.DataFrame(ss, columns = scaling_columns)

In [65]:
ss = standard_scaler.transform(X_valid[scaling_columns])
X_valid[scaling_columns] = pd.DataFrame(ss, columns = scaling_columns)

In [66]:
ss = standard_scaler.transform(X_test[scaling_columns])
X_test[scaling_columns] = pd.DataFrame(ss, columns = scaling_columns)

In [67]:
X_train = X_train.fillna(X_train.mean())
X_valid = X_valid.fillna(X_valid.mean())
X_test = X_test.fillna(X_test.mean())

In [68]:
X_train

,tweet_type,creator_follower_count,creator_following_count,engager_follower_count,engager_following_count,media,dt_dow,dt_hour,len_domains,creator_main_language,engager_main_language,creator_and_engager_have_same_main_language,is_tweet_in_creator_main_language,is_tweet_in_engager_main_language,engager_feature_number_of_previous_reply_engagement,creator_feature_number_of_previous_reply_engagement,number_of_engagements_positive,creator_number_of_engagements_positive,engager_number_of_engagements_ratio_reply,creator_number_of_engagements_ratio_reply
0,3,-0.168341,-0.091460,-0.021594,0.431057,4,-1.022717,0.443646,-0.339718,9,35,0,0,0,-0.252624,-0.185463,0.509384,-0.240262,0.000000,0.145833
1,0,-0.167882,0.000084,-0.030005,-0.176057,1,-1.022717,-0.443985,-0.339718,1,1,1,1,1,-0.252624,-0.194315,-0.394315,-0.241686,0.000000,0.000000
2,3,-0.168428,-0.106462,-0.044336,-0.239298,0,0.017867,0.443646,-0.339718,5,4,0,1,0,-0.252624,-0.194315,-0.376973,-0.241979,0.000000,0.000000
3,3,-0.142296,-0.046996,2.249159,9.416108,0,-1.543009,-1.479555,2.560201,12,1,0,1,0,0.779241,-0.193050,0.536360,-0.241435,0.049383,0.050000
4,3,0.104632,-0.077751,-0.043090,-0.036927,0,0.017867,0.591585,-0.339718,0,13,0,1,0,-0.080646,-0.110856,-0.332655,-0.144634,0.114286,0.028314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4338901,3,-0.168411,-0.101381,0.101986,0.722569,1,1.578743,-1.183678,-0.339718,1,1,1,1,1,-0.252624,-0.193050,-0.326874,-0.241016,0.000000,0.033333
4338902,0,-0.168441,-0.082681,-0.064379,0.002825,0,-1.543009,-1.775432,-0.339718,0,0,1,1,1,-0.209630,-0.194315,-0.267142,-0.242273,0.014493,0.000000
4338903,3,-0.155809,-0.107213,-0.077360,-0.397702,4,0.017867,-1.775432,-0.339718,17,3,0,0,1,-0.252624,-0.144998,-0.398168,-0.098348,0.000000,0.011350
4338904,3,-0.166943,-0.099788,-0.083176,-0.345302,4,-0.502425,-0.148108,-0.339718,0,0,1,1,1,-0.252624,-0.194315,-0.392388,-0.241686,0.000000,0.000000


## Modeling

In [75]:
model = Sequential([
    Dense(16, activation = 'relu', input_dim = X_train.shape[1]),
    Dense(8, activation = 'relu'),
    Dense(4, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [76]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy', # softmax : sparse_categorical_crossentropy, sigmoid : binary_crossentropy
    metrics=['binary_crossentropy']) # sigmoid :binary_crossentropy

In [77]:
result = model.fit(
    x = X_train,
    y = y_train,
    validation_data=(X_valid, y_valid),
    epochs=20,
    batch_size=32
)

Epoch 1/20
135591/135591 [==============================] - 59s 435us/step - loss: 0.1383 - binary_crossentropy: 0.1383 - val_loss: 0.1348 - val_binary_crossentropy: 0.1348
Epoch 2/20
135591/135591 [==============================] - 58s 430us/step - loss: 0.1332 - binary_crossentropy: 0.1332 - val_loss: 0.1348 - val_binary_crossentropy: 0.1348
Epoch 3/20
135591/135591 [==============================] - 57s 423us/step - loss: 0.1332 - binary_crossentropy: 0.1332 - val_loss: 0.1348 - val_binary_crossentropy: 0.1348
Epoch 4/20
135591/135591 [==============================] - 57s 421us/step - loss: 0.1332 - binary_crossentropy: 0.1332 - val_loss: 0.1348 - val_binary_crossentropy: 0.1348
Epoch 5/20
135591/135591 [==============================] - 57s 422us/step - loss: 0.1332 - binary_crossentropy: 0.1332 - val_loss: 0.1348 - val_binary_crossentropy: 0.1348
Epoch 6/20
135591/135591 [==============================] - 58s 425us/step - loss: 0.1332 - binary_crossentropy: 0.1332 - val_loss: 0.1

In [79]:
plt.plot(result.history['binary_crossentropy'])
plt.plot(result.history['val_binary_crossentropy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [68]:
model.evaluate(X_test, y_test)

27327/27327 [==============================] - 8s 291us/step - loss: 0.0865 - binary_crossentropy: 0.0865


[0.08646736294031143, 0.08646736294031143]

In [69]:
model.save(f'./saved_model/ffnn_{TARGET}')

INFO:tensorflow:Assets written to: ./saved_model/ffnn_reply/assets


## Predict

In [70]:
model = tf.keras.models.load_model(f'./saved_model/ffnn_{TARGET}')

In [80]:
pred = model.predict(X_test)

In [81]:
rce = compute_rce(pred, y_test)
rce

-0.00042965053279608867

In [74]:
average_precision_score(y_test, pred)

0.42356260668620216

In [76]:
X_test.columns

Index(['tweet_type', 'creator_follower_count', 'creator_following_count',
       'engager_follower_count', 'engager_following_count', 'media', 'dt_dow',
       'dt_hour', 'len_domains', 'creator_main_language',
       'engager_main_language', 'creator_and_engager_have_same_main_language',
       'is_tweet_in_creator_main_language',
       'is_tweet_in_engager_main_language',
       'engager_feature_number_of_previous_reply_engagement',
       'creator_feature_number_of_previous_reply_engagement',
       'number_of_engagements_positive',
       'number_of_engagements_positive_creator',
       'engager_number_of_engagements_ratio_reply',
       'number_of_engagements_ratio_reply'],
      dtype='object')